# Coleta de dados

In [1]:
# IMPORTAÇÃO DAS PRINCIPAIS BIBLIOTECAS PARA FUNCIONAMENTO DOS MODELOS:

In [2]:
import pandas as pd
import numpy as np
# import nltk
# import re
# from nltk.corpus import stopwords
# from nltk.stem import RSLPStemmer
# from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
# nltk.download('rslp')
# nltk.download('stopwords')
from  componetes_preprocessamento import RemoveStopWords, Cleaner, Tokenizador, Stemmer, Joiner, pega_resultados,salvando_em_arquivo
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\arm4\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arm4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# !conda install nltk
# import nltk
# nltk.download_gui()

In [4]:
dataset = pd.read_csv("datasets/reviews.csv")

# Pré-Processamento dos Dados

## Definindo Tipos

In [5]:
dataset.head(5)

,Unnamed: 0,order_id,review_id,review_score,review_comment_message
0,3,658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,1,Recebi bem antes do prazo estipulado.
1,4,8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,1,Parabéns lojas lannister adorei comprar pela I...
2,9,b9bf720beb4ab3728760088589c62129,8670d52e15e00043ae7de4c01cc2fe06,0,aparelho eficiente. no site a marca do aparelh...
3,12,9d6f15f95d01e79bd1349cc208361f09,4b49719c8a200003f700d3d986ea1a19,0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,15,e51478e7e277a83743b6f9991dbfa3fb,3948b09f7c818e2d86c9a546758b2335,1,"Vendedor confiável, produto ok e entrega antes..."


In [6]:
X = dataset["review_comment_message"].copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 199)

In [8]:
resultados = []

# Accuracy

In [9]:
modelo = RandomForestClassifier()
param_grid = { 
    'n_estimators':np.arange(1,26,2),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("RandonForest", GridSearchCV(modelo, param_grid, scoring="accuracy", cv=kfold, n_jobs=-1)),
                    ])

pipeline.fit(X_train, y_train)



Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('RandonForest',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                              estimator=RandomForestClassifier(), n_jobs=-1,
                              param_grid={'criterion': ['gini', 'entropy'],
                                          'max_features': ['auto', 'sqrt',
                                                           'log2'],
                                          'n_estimators': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                              scoring='accuracy'))])

In [10]:

cvres = pipeline["RandonForest"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.7190943484158023 {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 25}
0.7189543661828719 {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 25}
0.718115657419437 {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 25}
0.7179058306227194 {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 21}
0.7178358028680847 {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 21}
0.7178009966070613 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 23}
0.7177661170696987 {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 23}
0.7173814651408579 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 25}
0.7171369175721789 {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 19}
0.7169971918064352 {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 23}
0.716822085781679 {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 21}
0.7163330517079372 {'criterion': 'gini', 'max_

In [11]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("RF", "Grid Search", y_test,y_pred , "Acuracia", pipeline["RandonForest"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.72      0.84      0.77      3208
           0       0.50      0.21      0.30      2918
           1       0.76      0.90      0.82      6137

    accuracy                           0.72     12263
   macro avg       0.66      0.65      0.63     12263
weighted avg       0.69      0.72      0.68     12263

['RF', 'Grid Search', 0.7207045584277909, 0.631017159338, 0.6602326061553194, 0.6494945105965877, 'Acuracia', {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 25}]


# F1 - Score

In [12]:
modelo = RandomForestClassifier()
param_grid = { 
    'n_estimators': np.arange(1,26,2),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("RandonForest", GridSearchCV(modelo, param_grid, scoring="f1_macro", cv=kfold, n_jobs=-1)),
                    ])

pipeline.fit(X_train, y_train)



Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('RandonForest',
                 GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                              estimator=RandomForestClassifier(), n_jobs=-1,
                              param_grid={'criterion': ['gini', 'entropy'],
                                          'max_features': ['auto', 'sqrt',
                                                           'log2'],
                                          'n_estimators': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25])},
                              scoring='f1_macro'))])

In [13]:

cvres = pipeline["RandonForest"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6330877356246216 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 25}
0.6304490707713851 {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 21}
0.6299738571377143 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 17}
0.6298155045264895 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 23}
0.6297868760513472 {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 17}
0.6292149666652342 {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 23}
0.6288168971997765 {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 25}
0.6286041437287506 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 19}
0.6285590550913398 {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 25}
0.6280848198685163 {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 15}
0.6280548367057897 {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 25}
0.6278835477801037 {'criterion': 'entropy', 'max_feature

In [14]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("RF", "Grid Search", y_test,y_pred , "F1score", pipeline["RandonForest"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.71      0.84      0.77      3208
           0       0.49      0.22      0.31      2918
           1       0.76      0.89      0.82      6137

    accuracy                           0.72     12263
   macro avg       0.65      0.65      0.63     12263
weighted avg       0.68      0.72      0.69     12263

['RF', 'Grid Search', 0.7176058060833401, 0.6324358511533551, 0.6546565036522061, 0.6502136584027268, 'F1score', {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 25}]


# Otimizando com o Randomized search:

## Acurácia

In [15]:
from sklearn.model_selection import RandomizedSearchCV

modelo = RandomForestClassifier()
param_grid = { 
    'n_estimators': np.arange(1,26,2),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("RandonForest", RandomizedSearchCV(modelo, param_grid, cv=kfold, scoring='accuracy', n_iter=10, n_jobs=-1 )),
                    
                    ])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('RandonForest',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                                    estimator=RandomForestClassifier(),
                                    n_jobs=-1,
                                    param_distributions={'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_features': ['auto',
                                                                          'sqrt',
                                                                          'log2'],
                                                  

In [16]:
cvres = pipeline["RandonForest"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.7172068598377515 {'n_estimators': 25, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.7165426586756375 {'n_estimators': 21, 'max_features': 'auto', 'criterion': 'entropy'}
0.7165077791382752 {'n_estimators': 23, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.7151446194039165 {'n_estimators': 19, 'max_features': 'auto', 'criterion': 'gini'}
0.7127330706620835 {'n_estimators': 17, 'max_features': 'log2', 'criterion': 'gini'}
0.7084688151777769 {'n_estimators': 11, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.7024571632628586 {'n_estimators': 9, 'max_features': 'auto', 'criterion': 'entropy'}
0.6979138470542056 {'n_estimators': 7, 'max_features': 'sqrt', 'criterion': 'gini'}
0.6978087809968073 {'n_estimators': 7, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.628430022683912 {'n_estimators': 1, 'max_features': 'auto', 'criterion': 'entropy'}


In [17]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("RF", "Randomized Search", y_test,y_pred , "Acuracia", pipeline["RandonForest"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.72      0.85      0.77      3208
           0       0.50      0.22      0.30      2918
           1       0.76      0.89      0.82      6137

    accuracy                           0.72     12263
   macro avg       0.66      0.65      0.63     12263
weighted avg       0.69      0.72      0.69     12263

['RF', 'Randomized Search', 0.7196444589415314, 0.6325799177369695, 0.6590826808664735, 0.6515137850330578, 'Acuracia', {'n_estimators': 25, 'max_features': 'sqrt', 'criterion': 'entropy'}]


## F1 score

In [18]:
modelo = RandomForestClassifier()
param_grid = { 
    'n_estimators': np.arange(1,26,2),
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}
kfold = StratifiedKFold(10,shuffle=True, random_state=1)
pipeline = Pipeline([("Cleaner", Cleaner()), 
                    ("Tokenizador", Tokenizador("portuguese")), 
                    ("RemoveStopWords", RemoveStopWords("portuguese")), 
                    ("Stemmer", Stemmer()), 
                    ("Joiner", Joiner()),
                    ("Tfidf", CountVectorizer()),
                    ("RandonForest", RandomizedSearchCV(modelo, param_grid, cv=kfold, scoring="f1_macro", n_iter=10, n_jobs=-1 )),
                    
                    ])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tokenizador', Tokenizador(lingua='portuguese')),
                ('RemoveStopWords', RemoveStopWords(lingua='portuguese')),
                ('Stemmer', Stemmer()), ('Joiner', Joiner()),
                ('Tfidf', CountVectorizer()),
                ('RandonForest',
                 RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
                                    estimator=RandomForestClassifier(),
                                    n_jobs=-1,
                                    param_distributions={'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_features': ['auto',
                                                                          'sqrt',
                                                                          'log2'],
                                                  

In [19]:
cvres = pipeline["RandonForest"].cv_results_
idx_args = np.argsort(cvres["mean_test_score"])[::-1]
for i in idx_args:
    print(cvres["mean_test_score"][i], cvres["params"][i])

0.6322066851846074 {'n_estimators': 21, 'max_features': 'sqrt', 'criterion': 'gini'}
0.6321407764935444 {'n_estimators': 25, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.6264293226398857 {'n_estimators': 19, 'max_features': 'sqrt', 'criterion': 'entropy'}
0.6258935617517782 {'n_estimators': 25, 'max_features': 'log2', 'criterion': 'gini'}
0.6204436993269304 {'n_estimators': 13, 'max_features': 'log2', 'criterion': 'entropy'}
0.6185074339822629 {'n_estimators': 11, 'max_features': 'auto', 'criterion': 'gini'}
0.6051256135772591 {'n_estimators': 7, 'max_features': 'log2', 'criterion': 'gini'}
0.6010480539120143 {'n_estimators': 5, 'max_features': 'log2', 'criterion': 'gini'}
0.5914158653156584 {'n_estimators': 3, 'max_features': 'auto', 'criterion': 'gini'}
0.5481336024312451 {'n_estimators': 1, 'max_features': 'log2', 'criterion': 'gini'}


In [20]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

resultado = pega_resultados("RF", "Randomized Search", y_test,y_pred , "F1score", pipeline["RandonForest"].best_params_)
resultados.append(resultado)
print(resultado)

              precision    recall  f1-score   support

          -1       0.71      0.84      0.77      3208
           0       0.50      0.23      0.32      2918
           1       0.76      0.88      0.82      6137

    accuracy                           0.72     12263
   macro avg       0.66      0.65      0.64     12263
weighted avg       0.69      0.72      0.69     12263

['RF', 'Randomized Search', 0.7171980755117019, 0.6354828569860328, 0.6572270726696398, 0.6518305236970897, 'F1score', {'n_estimators': 21, 'max_features': 'sqrt', 'criterion': 'gini'}]


In [21]:
salvando_em_arquivo("resultados/RF_resultados.csv", resultados)